## 네이버영화 크롤링

In [1]:
import pandas as pd, numpy as np, scipy as sp, seaborn as sns
import matplotlib.pyplot as plt
import re
from bs4 import BeautifulSoup
from urllib.request import urlopen
import time
import random
%matplotlib inline

#### 함수정의

In [2]:
#메인페이지
def get_info_main(soup_info):
    i = 0
    m_rank_v =1

    while i<50:
        try:
            d_date_v = str(soup_info.find_all('p', class_='r_date')[0].get_text().strip().replace('.',''))

            m_title_v = soup_info.find_all('div', class_='tit5')[i].get_text().strip()

            rate_netizen_v = soup_info.find_all('td', class_='point')[i].get_text()

            m_url_raw_v = soup_info.find_all('div', class_='tit5')[i]
            m_url_raw_v = m_url_raw_v.a['href']
            m_url_v = (url_base + m_url_raw_v).strip()

            #리스트에 append
            d_date.append(d_date_v)
            m_title.append(m_title_v)
            m_rank.append(m_rank_v)
            rate_netizen.append(rate_netizen_v)
            m_url.append(m_url_v)        

        except:
            print('Under rank 50')
            break

        #print(i)
        m_rank_v += 1
        i += 1

In [3]:
#서브페이지
def get_info_sub(num_info):
    i_url = 0

    for i_url in range(0,len(df['m_url'])):
        html_sub = urlopen(df['m_url'][i_url])
        soup_sub = BeautifulSoup(html_sub, 'html.parser')    

        #관람객 평점
        rate_viewer_raw = soup_sub.find_all('a', class_='ntz_score')
        if len(rate_viewer_raw)>0:
            rate_viewer_raw = rate_viewer_raw[0].get_text().strip()
            if rate_viewer_raw == '관람객 평점 없음':
                rate_viewer = np.nan
                #print('1:',rate_viewer)
            else:
                rate_viewer = re.findall('\S(\d.\d{2})', rate_viewer_raw)[0]
                #print('1:',rate_viewer)
        else:
            rate_viewer = np.nan
            #print('1:',rate_viewer)

        #전문가 평점
        rate_spc_raw = soup_sub.find_all('div', class_='spc_score_area')
        if len(rate_spc_raw)>0:
            rate_spc_raw = rate_spc_raw[0].get_text().strip()
            if rate_spc_raw == '전문가 평점 없음':
                rate_spc = np.nan
                #print('3:',rate_spc)
            else:
                rate_spc = re.findall('\s(\d.\d{2})', rate_spc_raw)[0]
                #print('2:',rate_spc)
        else:
            rate_spc = np.nan
            #print('2:',rate_spc)

        #개요(장르)
        m_genre_raw = soup_sub.find_all('dl', class_='info_spec')
        if len(rate_spc_raw)>0:
            m_genre_raw = m_genre_raw[0].find_all('dd')[0].find_all('span')[0].get_text()
            m_genre = m_genre_raw.replace('\n','').replace('\t','').replace('\r','').strip()
            #print('3:',m_genre)
        else:
            m_genre = np.nan
            #print('3:',m_genre)

        #국가
        m_country_raw = soup_sub.find_all('dl', class_='info_spec')
        if len(m_country_raw)>0:
            m_country_raw = m_country_raw[0].find_all('dd')[0].find_all('span')[1].get_text()
            m_country = m_country_raw.replace('\n','').replace('\t','').replace('\r','').strip()
            #print('4:',m_country)
        else:
            m_country = np.nan
            #print('4:',m_country)

        #러닝타임   
        m_time_raw = soup_sub.find_all('dl', class_='info_spec')
        if len(m_time_raw)>0:
            m_time_raw = m_time_raw[0].find_all('dd')[0].find_all('span')[2].get_text()
            m_time = m_time_raw.replace('\n','').replace('\t','').replace('\r','').strip()
            #print('5:',m_time)
        else:
            m_time = np.nan
            #print('5:',m_time)

        #개봉일
        m_release_raw = soup_sub.find_all('dl', class_='info_spec')
        if len(m_release_raw)>0:
            m_release_raw = m_release_raw[0].find_all('dd')[0].find_all('span')[3].get_text()
            m_release_raw = m_release_raw.replace('\n','').replace('\t','').replace('\r','').strip()
            if '재개봉' in m_release_raw:
                m_release = m_release_raw.replace(',  ',' / ')
                #print('6:',m_release)
            else:
                m_release = re.findall('\d{4}.\d{2}.\d{2}', m_release_raw)[0]
                #print('6:',m_release)
        else:
            m_release = np.nan
            #print('6:',m_release)

        #감독
        m_director_raw = soup_sub.find_all('dl', class_='info_spec')
        if len(m_director_raw)>0:
            m_director_raw = m_director_raw[0].find_all('dd')[1].find_all('p')[0].get_text()
            m_director = m_director_raw.replace('\n','').replace('\t','').replace('\r','').strip()
            #print('7:',m_director)
        else:
            m_director = np.nan
            #print('7:',m_director)

        #누적관객수
        m_num_views_raw = soup_sub.find_all('p', class_='count')
        if len(m_num_views_raw)>0:
            m_num_views_raw = m_num_views_raw[0].get_text()
            m_num_views = int(m_num_views_raw.split('명')[0].replace(',',''))
            m_num_views_date = m_num_views_raw.split('명')[1].split(' ')[0].replace('(','')  
            #print('8:',m_num_views)
            #print('9:',m_num_views_date)
        else:
            m_num_views = np.nan
            m_num_views_date = np.nan
            #print('8:',m_num_views)
            #print('9:',m_num_views_date)
            
        #데이터프레임에 집어넣기
        df.loc[i_url,'rate_viewer'] = rate_viewer 
        df.loc[i_url,'rate_spc'] = rate_spc
        df.loc[i_url,'m_genre'] = m_genre
        df.loc[i_url,'m_country'] = m_country
        df.loc[i_url,'m_time'] = m_time
        df.loc[i_url,'m_release'] = m_release
        df.loc[i_url,'m_director'] = m_director
        df.loc[i_url,'m_num_views'] = m_num_views
        df.loc[i_url,'m_num_views_date'] = m_num_views_date
        
        #print('Success:', i_url, end=' / ')
        time.sleep(random.random()+random.random()*random.random())
    print('Success: ', num_info, end=' / ')

#### 실행

In [4]:
df_list = []

pr_m = pd.period_range(start = '20200427', end = None, periods = 3, freq = 'D')           
date_list = [v.strftime('%Y%m%d') for v in pr_m]

for i_date in date_list:
    url_base = 'https://movie.naver.com'
    url_sub = '/movie/sdb/rank/rmovie.nhn?sel=cur&tg=0&date={}'.format(i_date)
    page = url_base+url_sub
    html = urlopen(page)
    soup = BeautifulSoup(html, 'html.parser')

    d_date = []
    m_title = []
    m_rank = []
    rate_netizen = []
    rate_viewer = []
    rate_spc = []
    m_url = []

    #메인페이지
    get_info_main(soup)

    df = pd.DataFrame({'d_date':d_date, 'm_title':m_title, 'm_rank':m_rank,
                       'rate_netizen':rate_netizen, 'm_url':m_url})
    #서브페이지
    get_info_sub(i_date)
    
    #df_list에 집어넣기
    result_dict = df.to_dict()
    df_list.append(result_dict)
    del df


df_list = [pd.DataFrame(v) for v in df_list]
df_movie = pd.concat(df_list)

new_order = ['d_date','m_rank','m_title',
             'rate_netizen','rate_viewer','rate_spc',
             'm_genre','m_country','m_time','m_release','m_director',
             'm_num_views','m_num_views_date','m_url']

df_movie = df_movie[new_order].reset_index(drop=True)

df_movie.shape

Success:  20200427 / Success:  20200428 / Success:  20200429 / 

(150, 14)

#### 저장

In [5]:
# df_movie.head()
# df_movie.to_excel('/Users/hyungkeun/Desktop/네이버영화 크롤링/naver_movie_20200427-20200429.xlsx', index=False)